# NMSI–π*–γ_diss–e* Demo (2D pseudo-spectral)
This notebook runs a small 2D incompressible Navier–Stokes demo with the NMSI augmentation:
- π* cyclic forcing (zero-mean over periods)
- γ_diss spectral Z-window dissipation
- e* exponential stabilizer

It produces Energy $E(t)$, Enstrophy $\Omega(t)$, and max vorticity time series.

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# ---- Grid & spectral operators (2D periodic) ----
Nx = Ny = 64
Lx = Ly = 2*np.pi
dx = Lx/Nx
dy = Ly/Ny
x = np.arange(Nx)*dx
y = np.arange(Ny)*dy
X, Y = np.meshgrid(x, y, indexing='ij')

kx = np.fft.fftfreq(Nx, d=dx/(2*np.pi))
ky = np.fft.fftfreq(Ny, d=dy/(2*np.pi))
KX, KY = np.meshgrid(kx, ky, indexing='ij')
K2 = KX**2 + KY**2
K2[0,0] = 1.0  # avoid divide by zero for projection

def fft2(u):  return np.fft.rfftn(u, axes=(0,1)) if u.ndim==2 else np.fft.rfftn(u, axes=(0,1))
def ifft2(uh): return np.fft.irfftn(uh, s=(Nx,Ny), axes=(0,1))

def project_div_free(u):
    # Helmholtz projection in Fourier space for incompressibility
    uhx = np.fft.rfftn(u[...,0], axes=(0,1))
    uhy = np.fft.rfftn(u[...,1], axes=(0,1))
    kxr = KX[:, :uhx.shape[1]]
    kyr = KY[:, :uhx.shape[1]]
    K2r = (kxr**2 + kyr**2)
    K2r[0,0] = 1.0
    div_hat = kxr*uhx + kyr*uhy
    uhx_proj = uhx - kxr*div_hat/K2r
    uhy_proj = uhy - kyr*div_hat/K2r
    ux = np.fft.irfftn(uhx_proj, s=(Nx,Ny), axes=(0,1))
    uy = np.fft.irfftn(uhy_proj, s=(Nx,Ny), axes=(0,1))
    return np.stack([ux, uy], axis=-1)

def grad(u):
    # spectral derivatives
    uhx = np.fft.rfftn(u[...,0], axes=(0,1))
    uhy = np.fft.rfftn(u[...,1], axes=(0,1))
    kxr = KX[:, :uhx.shape[1]]
    kyr = KY[:, :uhx.shape[1]]
    ux_x = np.fft.irfftn(1j*kxr*uhx, s=(Nx,Ny), axes=(0,1))
    ux_y = np.fft.irfftn(1j*kyr*uhx, s=(Nx,Ny), axes=(0,1))
    uy_x = np.fft.irfftn(1j*kxr*uhy, s=(Nx,Ny), axes=(0,1))
    uy_y = np.fft.irfftn(1j*kyr*uhy, s=(Nx,Ny), axes=(0,1))
    return ux_x, ux_y, uy_x, uy_y

def nonlinear_term(u):
    ux_x, ux_y, uy_x, uy_y = grad(u)
    advx = u[...,0]*ux_x + u[...,1]*ux_y
    advy = u[...,0]*uy_x + u[...,1]*uy_y
    return -np.stack([advx, advy], axis=-1)

def laplacian(u):
    uhx = np.fft.rfftn(u[...,0], axes=(0,1))
    uhy = np.fft.rfftn(u[...,1], axes=(0,1))
    k2r = (KX[:, :uhx.shape[1]]**2 + KY[:, :uhx.shape[1]]**2)
    lx = np.fft.irfftn(-k2r*uhx, s=(Nx,Ny), axes=(0,1))
    ly = np.fft.irfftn(-k2r*uhy, s=(Nx,Ny), axes=(0,1))
    return np.stack([lx, ly], axis=-1)

def vorticity(u):
    ux_x, ux_y, uy_x, uy_y = grad(u)
    return uy_x - ux_y

def energy(u):    return 0.5*np.mean((u**2).sum(-1))
def enstrophy(u): return np.mean(vorticity(u)**2)
def wmax(u):      return np.abs(vorticity(u)).max()

# ---- NMSI operators ----
A_pi, omega_pi, phi = 0.2, 2.0, 0.0
lam_e, alpha_e      = 0.4, 0.2
gamma0              = 0.6
nu                  = 1e-3

def Z_window_k(uh):
    # build kmag on rfft grid
    kxr = KX[:, :uh.shape[1]]
    kyr = KY[:, :uh.shape[1]]
    kmag = np.sqrt(kxr**2 + kyr**2)
    kmax = kmag.max()
    Z = (kmag >= 0.6*kmax) & (kmag <= 0.9*kmax)
    return Z.astype(float)

def step(u, t, dt):
    NL = nonlinear_term(u)
    visc = nu*laplacian(u)
    Fpi = A_pi*np.sin(omega_pi*t + phi)*u  # zero-mean over cycles
    
    uhx = np.fft.rfftn(u[...,0], axes=(0,1))
    uhy = np.fft.rfftn(u[...,1], axes=(0,1))
    Z = Z_window_k(uhx)
    uhx_new = uhx + dt*( np.fft.rfftn(NL[...,0]+visc[...,0]+Fpi[...,0], axes=(0,1))
                        - (gamma0*Z)*uhx - lam_e*np.exp(-alpha_e*t)*uhx )
    uhy_new = uhy + dt*( np.fft.rfftn(NL[...,1]+visc[...,1]+Fpi[...,1], axes=(0,1))
                        - (gamma0*Z)*uhy - lam_e*np.exp(-alpha_e*t)*uhy )
    ux = np.fft.irfftn(uhx_new, s=(Nx,Ny), axes=(0,1))
    uy = np.fft.irfftn(uhy_new, s=(Nx,Ny), axes=(0,1))
    unew = np.stack([ux, uy], axis=-1)
    return project_div_free(unew)

# ---- Initial condition: 2D Taylor–Green-like ----
u0 = np.zeros((Nx,Ny,2))
u0[...,0] =  np.sin(X)*np.cos(Y)
u0[...,1] = -np.cos(X)*np.sin(Y)
u0 = project_div_free(u0)

# ---- Time loop ----
dt = 2.5e-3
t_end = 2.0
nt = int(t_end/dt)
u = u0.copy()

E_hist, Om_hist, W_hist, T = [], [], [], []

for n in range(nt):
    t = n*dt
    u = step(u, t, dt)
    if n % 20 == 0:
        E_hist.append(energy(u))
        Om_hist.append(enstrophy(u))
        W_hist.append(wmax(u))
        T.append(t)

E_hist = np.array(E_hist); Om_hist = np.array(Om_hist); W_hist = np.array(W_hist); T = np.array(T)

fig, axs = plt.subplots(3,1, figsize=(6,8))
axs[0].plot(T, E_hist); axs[0].set_ylabel('Energy E(t)')
axs[1].plot(T, Om_hist); axs[1].set_ylabel('Enstrophy Ω(t)')
axs[2].plot(T, W_hist); axs[2].set_ylabel('max|ω|'); axs[2].set_xlabel('t')
plt.tight_layout()
plt.show()

print("Final diagnostics: E={:.4e}, Ω={:.4e}, max|ω|={:.4e}".format(E_hist[-1], Om_hist[-1], W_hist[-1]))
